In [2]:
import cv2
import numpy as np
from PIL import ImageGrab

In [3]:

def filter_region(image, vertices):
    """
    Create the mask using the vertices and apply it to the input image
    """
    mask = np.zeros_like(image)
    cv2.imshow("image in filter",image)
    if len(mask.shape)==2:
        cv2.fillPoly(mask, vertices, 255)
    else:
        cv2.fillPoly(mask, vertices, (255,)*mask.shape[2]) # in case, the input image has a channel dimension   
        print(image)
        print(mask)
    return cv2.bitwise_and(image, mask)

In [4]:
def select_region(image):
    """
    It keeps the region surrounded by the `vertices` (i.e. polygon).  Other area is set to 0 (black).
    """
    # first, define the polygon by vertices
    rows, cols = image.shape[:2]
    bottom_left  = [cols*0.03, rows*0.72]
    top_left     = [cols*0.4, rows*0.45]
    bottom_right = [cols*0.95, rows*0.78]
    top_right    = [cols*0.5, rows*0.45] 
    # the vertices are an array of polygons (i.e array of arrays) and the data type must be integer
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    return filter_region(image, vertices)


def detected_edges(image, low_threshold=50, high_threshold=150):
    return cv2.Canny(image, low_threshold, high_threshold)

In [ ]:
video = cv2.VideoCapture("videoplayback.mp4")

while True:
    ret, frame = video.read()
    if not ret:
        video = cv2.VideoCapture("videoplayback.mp4")
        continue
    frame = cv2.GaussianBlur(frame, (5, 5), 0)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #lower_white = np.array([0,0,150], dtype=np.uint8)
    #upper_white = np.array([255,40,255], dtype=np.uint8)
    
    #mask = cv2.inRange(hsv, lower_white, upper_white)
    
    edges = cv2.Canny(frame, 75, 150)
    image =select_region(edges)
    lines = cv2.HoughLinesP(image, 1, np.pi/180, 50, maxLineGap=300)
    

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (250, 255, 150), 2)
    cv2.imshow("frame", frame)
    cv2.imshow("edges", image)
    #cv2.imshow("mask",mask)
    key = cv2.waitKey(25)
    if key == 27:
        cv2.destroyAllWindows()
        break  
video.release()
